## Gregory Regression

In this notebook, I will be performing gregory regression. This involves finding the linear relationship between N = F - αΔT, where N = change in net TOA (top of atmosphere) radiative flux, F = a constant that represents how the climate responds to a radiative perturbation measured in Wm^-2, ΔT = change in TAS (surface temperature). 

We derive our best estimates of -α (slope) and F (intercept) through OLS regression of N against ΔT.


In [1]:
#imports
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from scipy import stats
import numpy as np
import pandas as pd
import xarray as xr
import zarr
from sklearn.linear_model import LinearRegression

In [54]:
#open processed data
ds_tas_hist_ssp585 = xr.open_dataset('processed_tas_global_data.nc')
ds_tas_1pctCO2 = xr.open_dataset('processed_tas_global_1pctCO2_data.nc')
ds_toa_hist_ssp585 = xr.open_dataset('processed_toa_global_data.nc')
ds_toa_abrupt4xCO2 = xr.open_dataset('processed_abrupt4xCO2_toa_global_data.nc')

In [36]:
#converted the model objects to string types, but it doesn't help with the duplications
ds_tas_hist_ssp585['model'] = ds_tas_hist_ssp585['model'].astype('str')
ds_tas_1pctCO2['model'] = ds_tas_1pctCO2['model'].astype('str')
ds_toa_hist_ssp585['model'] = ds_toa_hist_ssp585['model'].astype('str')
ds_toa_abrupt4xCO2['model'] = ds_toa_abrupt4xCO2['model'].astype('str')

In [57]:
#slice time to be the same 
ds_tas_hist_ssp585 = ds_tas_hist_ssp585.isel(time=slice(0,1800))
ds_toa_hist_ssp585 = ds_toa_hist_ssp585.isel(time=slice(0,1800))

In [58]:
ds_toa_hist_ssp585

<xarray.Dataset>
Dimensions:  (time: 1800, model: 27)
Coordinates:
  * time     (time) datetime64[ns] 1850-01-31 1850-02-28 ... 1999-12-31
  * model    (model) object 'NorESM2-MM' 'AWI-CM-1-1-MR' ... 'CESM2'
Data variables:
    toa      (model, time) float64 ...
    cre      (model, time) float64 ...
    tas      (model, time) float64 ...

In [61]:
#have to consider they don't have the same models, have to join/merge together based on model names
#join inner keeps the lesser values 
ds_merged = xr.concat([ds_tas_hist_ssp585, ds_tas_1pctCO2, ds_toa_hist_ssp585, ds_toa_abrupt4xCO2], dim = 'model', coords = 'minimal', compat = 'override')

PROBLEMS: 
- Time differences (some only span from 1850-2000 (1pctCO2/abrupt4xCO2)) - solved with slicing
- duplicates in model names that aren't merging on that dimension because there are copies from the 4 datasets (if I fix time differences w coords, i could possibly fix this issue --> DUPLICATES DID NOT GET FIXED after having the same time

In [62]:
ds_merged

<xarray.Dataset>
Dimensions:  (time: 1800, model: 143)
Coordinates:
  * time     (time) datetime64[ns] 1850-01-31 1850-02-28 ... 1999-12-31
  * model    (model) object 'CNRM-CM6-1' 'GISS-E2-1-G' ... 'CNRM-CM6-1'
Data variables:
    tas      (model, time) float64 284.1 284.3 284.8 285.9 ... 293.0 292.1 291.6
    toa      (model, time) float64 nan nan nan nan ... 2.118 7.842 9.86 10.79
    cre      (model, time) float64 nan nan nan nan ... -24.85 -26.96 -28.52

In [ ]:
#net TOA = N variable

#net TAS = deltaT variable